In [ ]:
import grpc
from protos.nlp.dm import dm_pb2
from protos.nlp.dm import dm_pb2_grpc

def run():
    #本地测试
    # channel = grpc.insecure_channel('localhost:50051')
    #数字人
    # channel = grpc.insecure_channel('172.31.208.4:68999')
    #客服
    # channel = grpc.insecure_channel('172.31.208.4:50035')
    #全部服务
    channel = grpc.insecure_channel('172.31.208.4:55555')
    # 调用 rpc 服务
    stub = dm_pb2_grpc.DmStub(channel)
    history = {}
    while True:
        text = input()
        response = stub.GetDm(dm_pb2.DmRequest(text=text, history=history,session_id='1', robot_id = -1))
        print("Dm client received: " + response.answer)
        history = response.history
        print('history :', history.conversation, history.intent, history.slots, history.entities, history.task_id)
if __name__ == '__main__':
    run()

 碧昂斯的出生地


Dm client received: 美国德克萨斯州休斯顿
history : ['', '', '碧昂斯的出生地', '美国德克萨斯州休斯顿'] ['DocQa'] [] ['美国纽约州长岛杰里科', '德州休士顿', '乔治亚州亚特兰大'] []


 不对


Dm client received: 我想了想，是美国纽约州长岛杰里科吗？
history : ['碧昂斯的出生地', '美国德克萨斯州休斯顿', '不对', '我想了想，是美国纽约州长岛杰里科吗？'] ['DocQa', 'Mutil-Turn'] [] ['德州休士顿', '乔治亚州亚特兰大'] [-1]


 是的


Dm client received: 嘿嘿～，厉害吧
history : ['不对', '我想了想，是美国纽约州长岛杰里科吗？', '是的', '嘿嘿～，厉害吧'] ['DocQa', 'Mutil-Turn', 'Chat'] [] [] []
